# Instructions

This simulates retail sales for cigarettes, Juul and other e-cigarettes. We are interested in what conclusions you draw from this data about Juul's performance and market impact and how you visualize them. 

Feel free to use whatever means you prefer for your analysis and communication of it and its results. What you send back should be able to stand on its own, i.e., we should be able to understand it without requiring additional narration.

Please only use RMA and ignore CRMA and SRMA geographies for this analysis.

# Data exploration

In [1]:
import pandas as pd
import numpy as np
import os 
import sys
import re
import datetime
os.listdir()

['.DS_Store',
 '.git',
 '.ipynb_checkpoints',
 'data_exploration.ipynb',
 'draft.ipynb',
 'final.ipynb',
 'market_share.csv',
 'notebook.tex',
 'output_11_0.png',
 'output_13_0.png',
 'output_15_0.png',
 'output_17_0.png',
 'output_19_0.png',
 'output_19_1.png',
 'output_21_0.png',
 'output_21_1.png',
 'output_28_0.png',
 'output_31_0.png',
 'output_33_0.png',
 'output_9_0.png',
 'sales.csv',
 'sales_new.csv',
 'sku.csv',
 'sku_new.csv',
 'Untitled.ipynb']

In [2]:
sales = pd.read_csv('sales.csv'); sales.head()

,Geography,Product,Time,Dollar Sales,Unit Sales,SKU
0,Circle K Florida-RMA - Conv,CIGARETTES,4 Weeks Ending 01-31-16,28921840.49,4968512.070,Cigarettes Total
1,Circle K Florida-RMA - Conv,CIGARETTES,4 Weeks Ending 02-28-16,30276220.80,5139634.753,Cigarettes Total
2,Circle K Florida-RMA - Conv,CIGARETTES,4 Weeks Ending 03-27-16,31535167.82,5366848.000,Cigarettes Total
3,Circle K Florida-RMA - Conv,CIGARETTES,4 Weeks Ending 04-24-16,31693487.95,5420033.091,Cigarettes Total
4,Circle K Florida-RMA - Conv,CIGARETTES,4 Weeks Ending 05-22-16,31390945.73,5380230.139,Cigarettes Total


In [3]:
sku = pd.read_csv('sku.csv'); sku.head()

,SKU Legend,Unit Sales,SKU
0,JUUL-ELECTRONIC SMOKING DEVICES,–,JUUL Total
1,CIGARETTES,"75,083,502,411",Cigarettes Total
2,ELECTRONIC SMOKING DEVICES,"939,305,632",E-Cigs Total
3,JUUL ELCTRNC SMKNG ACSRY MIINT DISPOSABLE 4 CT...,"27,211,643",JUUL Refill Kits
4,JUUL ELCTRNC SMKNG ACSRY BRUULE DISPOSABLE 4 C...,"7,979,019",JUUL Refill Kits


## Removoing non RMA Geos

As per instructions

In [4]:
# filtering sales to only RMA geos
sales = sales[np.array(sales.Geography.apply(lambda x: re.search("-RMA", x))) != None]

## Fixing Date Strings

Removing the "4 weeks ending" string and creating a start and end date column for each sale

In [5]:
# is it always ending in 4 weeks?
print(sales.shape == 
      sales[np.array(sales.Time.apply(lambda x: re.search("4 Weeks Ending", x))) != None].shape) #yes

sales['end_time'] = sales.Time.apply(lambda t: re.split(" ", t)[-1])
sales.end_time = sales.end_time.apply(lambda x: datetime.datetime.strptime(x, "%m-%d-%y"))
sales['start_time'] = sales.end_time.apply(lambda x: x - datetime.timedelta(weeks = 4))

True


## Simplifying SKU Legend Names using Tf-Idf

Using Tf-Idf to extract the important words from each legend name

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
sku['sku_id'] = sku.index
sku['unit_sales'] = sku.loc[:,['Unit Sales']]
sku['sku_legend'] = sku.loc[:,['SKU Legend']]
sku = sku.drop(labels = ['SKU Legend', 'Unit Sales'], axis = 1)
sku['sku_legend_text'] = sku.sku_legend.apply(lambda x: ''.join(re.findall("[A-Za-z|\s]", x)).lower())
# Simplifying SKU Names using TF-IDF
vectorizer = TfidfVectorizer()
response = vectorizer.fit_transform(list(sku.sku_legend_text))
sku_dtm = pd.DataFrame(response.toarray().transpose(), index = vectorizer.get_feature_names())
sku_simple = []
for sku_id in list(sku.sku_id):
    tf = sku_dtm.iloc[:,sku_id]
    tf = sku_dtm.iloc[:,sku_id]
    tf = tf[tf != 0]
    sku_simple.append([sku_id, ' '.join(list(tf.sort_values()[-3:].index))])
sku_simple = pd.DataFrame(sku_simple); sku_simple.columns = ['sku_id', 'sku_tf_idf']
sku = sku.merge(sku_simple,how = 'inner', on = 'sku_id')
sku

,SKU,sku_id,unit_sales,sku_legend,sku_legend_text,sku_tf_idf
0,JUUL Total,0,–,JUUL-ELECTRONIC SMOKING DEVICES,juulelectronic smoking devices,devices smoking juulelectronic
1,Cigarettes Total,1,"75,083,502,411",CIGARETTES,cigarettes,cigarettes
2,E-Cigs Total,2,"939,305,632",ELECTRONIC SMOKING DEVICES,electronic smoking devices,electronic devices smoking
3,JUUL Refill Kits,3,"27,211,643",JUUL ELCTRNC SMKNG ACSRY MIINT DISPOSABLE 4 CT...,juul elctrnc smkng acsry miint disposable ct,elctrnc disposable miint
4,JUUL Refill Kits,4,"7,979,019",JUUL ELCTRNC SMKNG ACSRY BRUULE DISPOSABLE 4 C...,juul elctrnc smkng acsry bruule disposable ct,elctrnc disposable bruule
5,JUUL Refill Kits,5,"7,395,437",JUUL ELCTRNC SMKNG ACSRY FRUUT DISPOSABLE 4 CT...,juul elctrnc smkng acsry fruut disposable ct,elctrnc disposable fruut
6,JUUL Refill Kits,6,"6,871,591",JUUL ELCTRNC SMKNG ACSRY MANGO DISPOSABLE 4 CT...,juul elctrnc smkng acsry mango disposable ct,elctrnc disposable mango
7,JUUL Refill Kits,7,"9,532,052",JUUL ELCTRNC SMKNG ACSRY TABAAC DISPOSABLE 4 C...,juul elctrnc smkng acsry tabaac disposable ct,elctrnc disposable tabaac
8,JUUL Devices,8,"5,596,081",JUUL ELECTRONIC SMKNG DVC ELECTRONIC CIGRTT KT...,juul electronic smkng dvc electronic cigrtt kt...,kt rechargeable electronic
9,JUUL Devices,9,"1,533,344",JUUL ELECTRONIC SMKNG DVC ELECTRONIC CIGRTT KT...,juul electronic smkng dvc electronic cigrtt kt...,kt rechargeable electronic


In [8]:
sales.head()

,Geography,Product,Time,Dollar Sales,Unit Sales,SKU,end_time,start_time
0,Circle K Florida-RMA - Conv,CIGARETTES,4 Weeks Ending 01-31-16,28921840.49,4968512.070,Cigarettes Total,2016-01-31,2016-01-03
1,Circle K Florida-RMA - Conv,CIGARETTES,4 Weeks Ending 02-28-16,30276220.80,5139634.753,Cigarettes Total,2016-02-28,2016-01-31
2,Circle K Florida-RMA - Conv,CIGARETTES,4 Weeks Ending 03-27-16,31535167.82,5366848.000,Cigarettes Total,2016-03-27,2016-02-28
3,Circle K Florida-RMA - Conv,CIGARETTES,4 Weeks Ending 04-24-16,31693487.95,5420033.091,Cigarettes Total,2016-04-24,2016-03-27
4,Circle K Florida-RMA - Conv,CIGARETTES,4 Weeks Ending 05-22-16,31390945.73,5380230.139,Cigarettes Total,2016-05-22,2016-04-24


In [9]:
sku.to_csv('sku_new.csv', index = False)
sales.to_csv('sales_new.csv', index = False)